## bringing FASTA files (barcode and sequences) and metadata table in to demultiplex 
## reading them in from a folder on the desktop (no spaces allowed in names), have to be the only files in the folder, and have to be titled "sequences" and "barcodes" 
## output path is the same as input but with .qza on the end 

In [17]:
## importing files from a desktop folder and converting to a .qza
! qiime tools import \
--type EMPSingleEndSequences \
--input-path /Users/madiapgar/Desktop/qiimework/MG \
--output-path /Users/madiapgar/Desktop/qiimework/MG.qza

Imported /Users/madiapgar/Desktop/qiimework/MG as EMPSingleEndDirFmt to /Users/madiapgar/Desktop/qiimework/MG.qza


In [9]:
! pwd

/Users/madiapgar/neonatal-gut-microbiome


In [18]:
## now we can demultiplex everything
## i-seqs is the qza file we put the sequences and barcodes in 
## m-barcodes is the metadata file/table that we created via excel
## still need to include pathway to files since they're on desktop and not in this directory
! qiime demux emp-single \
--i-seqs /Users/madiapgar/Desktop/qiimework/MG.qza \
--m-barcodes-file /Users/madiapgar/Desktop/qiimework/MG-barcode.txt \
--m-barcodes-column barcode_sequence \
--o-per-sample-sequences demux_MG.qza \
--o-error-correction-details demux_MG-details.qza \
--p-no-golay-error-correction

Saved SampleData[SequencesWithQuality] to: demux_MG.qza
Saved ErrorCorrectionDetails to: demux_MG-details.qza


In [19]:
## creating the summarization of demultiplexing that I can actually look at 
! qiime demux summarize \
--i-data demux_MG.qza \
--o-visualization demux_MG.qzv

Saved Visualization to: demux_MG.qzv


In [20]:
! qiime tools view demux_MG.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

## feature/biom table construction via DADA2 

In [21]:
! qiime dada2 denoise-single \
--i-demultiplexed-seqs demux_MG.qza \
--p-trim-left 15 \
--p-trunc-len 200 \
--p-n-threads 2 \
--o-representative-sequences rep-seqs-MG-dada2.qza \
--o-table table-MG-dada2.qza \
--o-denoising-stats stats-MG-dada2.qza


Saved FeatureTable[Frequency] to: table-MG-dada2.qza
Saved FeatureData[Sequence] to: rep-seqs-MG-dada2.qza
Saved SampleData[DADA2Stats] to: stats-MG-dada2.qza


In [22]:
! qiime metadata tabulate \
--m-input-file stats-MG-dada2.qza \
--o-visualization stats-MG-dada2.qzv

Saved Visualization to: stats-MG-dada2.qzv


In [23]:
! qiime tools view stats-MG-dada2.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [24]:
!mv rep-seqs-MG-dada2.qza rep-seqs-MG.qza
!mv table-MG-dada2.qza table-MG.qza

In [25]:
## mapping of feature IDs to sequences, you can now put these into BLAST or PiCRUST2
! qiime feature-table tabulate-seqs \
--i-data rep-seqs-MG.qza \
--o-visualization rep-seqs-MG.qzv

Saved Visualization to: rep-seqs-MG.qzv


In [28]:
## visual summarization of feature table 
! qiime feature-table summarize \
--i-table table-MG.qza \
--o-visualization table-MG.qzv \
--m-sample-metadata-file /Users/madiapgar/Desktop/qiimework/MG-barcode.txt

Saved Visualization to: table-MG.qzv


In [1]:
! qiime tools view table-MG.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [30]:
! qiime tools view rep-seqs-MG.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

## filtering my feature table for a contaminated sample MG.73A and MG.73B 
## will be using actable-MG.qza from now on as my feature table!

In [3]:
## doing this via identifier-based filtering and only on my feature table and not my rep-seqs 
## need to generate a new metadata file that doesn't include the sample ID i want to filter out - this only has to 
## contain the sample ID column of the metadata table 
! qiime feature-table filter-samples \
--i-table table-MG.qza \
--m-metadata-file /Users/madiapgar/Desktop/qiimework/kept-samples-MG.txt \
--o-filtered-table actable-MG.qza

Saved FeatureTable[Frequency] to: actable-MG.qza


## below are additional steps not included in the tutorial that the lab uses for data sets
## building the phylogenic tree via sepp 

In [4]:
## last line denotes the silva 128 reference database that is used to construct the phylogenic tree 
! qiime fragment-insertion sepp \
--i-representative-sequences rep-seqs-MG.qza \
--o-tree tree-MG.qza \
--o-placements placements-MG.qza \
--i-reference-database /Users/madiapgar/Desktop/qiimework/sepp-refs-silva-128.qza

Saved Phylogeny[Rooted] to: tree-MG.qza
Saved Placements to: placements-MG.qza


In [2]:
## filtering of sepp phylogenic tree via feature table files to only include relevant ASVs
## this is the first step of filtering is based on what was put into the tree and what wasn't
## remember you need to use your filtered table that you generated above for this!
! qiime fragment-insertion filter-features \
--i-table actable-MG.qza \
--i-tree tree-MG.qza \
--o-filtered-table filtered-table-MG.qza \
--o-removed-table removed-table-MG.qza

Saved FeatureTable[Frequency] to: filtered-table-MG.qza
Saved FeatureTable[Frequency] to: removed-table-MG.qza


## taxonomic classification and filtering additional steps 
## this comes before alpha and beta diversity analysis

In [4]:
## doing taxonomic classification before alpha and beta diversity analysis, also includes an extra step to filter
## table based on taxonomic classification 
## classifier is pulled from the silva database and can be found under data resources on the qiime website
## your classifier needs to be up to date with the qiime version you're running or else it won't work
! qiime feature-classifier classify-sklearn \
--i-classifier /Users/madiapgar/Desktop/qiimework/silva-138-99-515-806-nb-classifier.qza \
--i-reads rep-seqs-MG.qza \
--o-classification taxonomy-MG.qza

Saved FeatureData[Taxonomy] to: taxonomy-MG.qza


In [5]:
! qiime metadata tabulate \
--m-input-file taxonomy-MG.qza \
--o-visualization taxonomy-MG.qzv

Saved Visualization to: taxonomy-MG.qzv


In [6]:
! qiime tools view taxonomy-MG.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [7]:
## filtering my taxonomic table 
! qiime taxa filter-table \
--i-table filtered-table-MG.qza \
--i-taxonomy taxonomy-MG.qza \
--p-include p_ \
--p-exclude mitochondria,chloroplast \
--o-filtered-table tax-filtered-MG.qza

Saved FeatureTable[Frequency] to: tax-filtered-MG.qza


In [8]:
## i'm creating a visualization for my taxonomic filtered table and I will be able to use that table for my alpha/beta
## diversity aanlysis and building my taxa bar chart 
! qiime feature-table summarize \
--i-table tax-filtered-MG.qza \
--o-visualization tax-filtered-MG.qzv

Saved Visualization to: tax-filtered-MG.qzv


In [12]:
! qiime tools view tax-filtered-MG.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

## diversity analysis - rareifying data and normalizing sample read numbers prior to alpha/beta diversity 


In [2]:
## pull out taxa barplot, faithpd, weighted and unweighted unifrac emperor plots for a presentation to show the
## collaboraters 
! qiime diversity core-metrics-phylogenetic \
--i-phylogeny tree-MG.qza \
--i-table tax-filtered-MG.qza \
--p-sampling-depth 28231 \
--m-metadata-file /Users/madiapgar/Desktop/qiimework/MG-metadata.txt \
--output-dir core-metrics-corrected-MG

Saved FeatureTable[Frequency] to: core-metrics-corrected-MG/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-corrected-MG/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-corrected-MG/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-corrected-MG/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-corrected-MG/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-corrected-MG/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-corrected-MG/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-corrected-MG/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-corrected-MG/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-corrected-MG/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-corrected-MG/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-corrected-MG/jaccard_pcoa_results.qza
Sa

## alpha diversity community richness (looks at the phylogenetic relationships between samples) 

In [3]:
## so apparently it works to edit the same metadata file used in the core diversity (above) and rerun further analyses 
! qiime diversity alpha-group-significance \
--i-alpha-diversity core-metrics-corrected-MG/faith_pd_vector.qza \
--m-metadata-file /Users/madiapgar/Desktop/qiimework/MG-metadata.txt \
--o-visualization core-metrics-corrected-MG/faith-pd-MG.qzv

Saved Visualization to: core-metrics-corrected-MG/faith-pd-MG.qzv


## alpha diversity community evenness (what are proportions of species in the samples and how are they related to each other?) 

In [5]:
! qiime diversity alpha-group-significance \
--i-alpha-diversity core-metrics-corrected-MG/evenness_vector.qza \
--m-metadata-file /Users/madiapgar/Desktop/qiimework/MG-metadata.txt \
--o-visualization core-metrics-corrected-MG/evenness-MG.qzv

Saved Visualization to: core-metrics-corrected-MG/evenness-MG.qzv


## beta diversity analysis based on mouse sex (male or female) 

In [5]:
! qiime diversity beta-group-significance \
--i-distance-matrix core-metrics-corrected-MG/unweighted_unifrac_distance_matrix.qza \
--m-metadata-file /Users/madiapgar/Desktop/qiimework/MG-metadata.txt \
--m-metadata-column sex \
--o-visualization core-metrics-corrected-MG/unweighted-unifrac-sex.qzv \
--p-pairwise

Saved Visualization to: core-metrics-corrected-MG/unweighted-unifrac-sex.qzv


## beta diversity analysis based on mouse age (this is the most important one) 

In [1]:
## this doesn't work bc age needs to be numerical and it's trying to treat it as categorical
## use MG-barcode.txt if you want to redo this output but its not needed 
! qiime diversity beta-group-significance \
--i-distance-matrix core-metrics-corrected-MG/unweighted_unifrac_distance_matrix.qza \
--m-metadata-file /Users/madiapgar/Desktop/qiimework/MG-metadata.txt \
--m-metadata-column age \
--o-visualization core-metrics-corrected-MG/unweighted-unifrac-age.qzv \
--p-pairwise

Saved Visualization to: core-metrics-corrected-MG/unweighted-unifrac-age.qzv


## beta diversity ordination (how samples changed over time based on age 7-14 days old) 

In [7]:
! qiime emperor plot \
--i-pcoa core-metrics-corrected-MG/unweighted_unifrac_pcoa_results.qza \
--m-metadata-file /Users/madiapgar/Desktop/qiimework/MG-metadata.txt \
--p-custom-axes age \
--o-visualization core-metrics-corrected-MG/unweighted-unifrac-emperor-age.qzv

! qiime emperor plot \
--i-pcoa core-metrics-corrected-MG/bray_curtis_pcoa_results.qza \
--m-metadata-file /Users/madiapgar/Desktop/qiimework/MG-metadata.txt \
--p-custom-axes age \
--o-visualization core-metrics-corrected-MG/bray-curtis-emperor-age.qzv

Saved Visualization to: core-metrics-corrected-MG/unweighted-unifrac-emperor-age.qzv
Saved Visualization to: core-metrics-corrected-MG/bray-curtis-emperor-age.qzv


## alpha rarefaction plotting (alpha diversity metrics at multiple sampling depths)

In [13]:
## used median frequency of samples for p-max-depth 
## this one did not need to be corrected
! qiime diversity alpha-rarefaction \
--i-table tax-filtered-MG.qza \
--i-phylogeny tree-MG.qza \
--p-max-depth 57654 \
--m-metadata-file /Users/madiapgar/Desktop/qiimework/MG-metadata.txt \
--o-visualization alpha-rarefaction-MG.qzv

Saved Visualization to: alpha-rarefaction-MG.qzv


## taxonomic bar plots!

In [14]:
## this one did not need to be corrected
! qiime taxa barplot \
--i-table tax-filtered-MG.qza \
--i-taxonomy taxonomy-MG.qza \
--m-metadata-file /Users/madiapgar/Desktop/qiimework/MG-metadata.txt \
--o-visualization taxa-barplots-MG.qzv

Saved Visualization to: taxa-barplots-MG.qzv


In [2]:
## taxa barplots (these are the most interesting)
! qiime tools view taxa-barplots-MG.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [16]:
## alpha rarefaction based on mouse sex (couldn't do age bc wasn't categorical)
! qiime tools view alpha-rarefaction-MG.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [1]:
## unweighted emperor plot based on age 
## this is updated
! qiime tools view core-metrics-corrected-MG/unweighted-unifrac-emperor-age.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [4]:
## weighted emperor plot based on age
## this is updated 
! qiime tools view core-metrics-corrected-MG/bray-curtis-emperor-age.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [7]:
## alpha diversity richness
## this is updated
! qiime tools view core-metrics-corrected-MG/faith-pd-MG.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [6]:
## alpha diversity evenness
## this is updated
! qiime tools view core-metrics-corrected-MG/evenness-MG.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [8]:
## regular weighted emperor plot (corrected)
! qiime tools view core-metrics-corrected-MG/weighted_unifrac_emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [9]:
## regular unweighted emperor plot (corrected)
! qiime tools view core-metrics-corrected-MG/unweighted_unifrac_emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [2]:
## beta group significance graph 
! qiime tools view core-metrics-MG/unweighted-unifrac-age-sig.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

## converting my biom table to .biom format and my rep-seqs to fasta format so I can put them in PiCRUST2

In [2]:
! qiime tools export \
    --input-path actable-MG.qza \
    --output-path picrust

! qiime tools export \
    --input-path rep-seqs-MG.qza \
    --output-path picrust

Exported actable-MG.qza as BIOMV210DirFmt to directory picrust
Exported rep-seqs-MG.qza as DNASequencesDirectoryFormat to directory picrust
